# Veri Yogun Uygulamalar Modul Projesi: Zaman serisi siniflandirma problemi


## ERTUĞRUL SERT TUBİTAK PROJE
Dr. Ismail Guzel

Milli Teknoloji Hamlesi altinda Yapay Zeka Uzmanlık Programı kapsamında 18 saatlik Veri Yogun Uygulamar egitimi sonunda bu projenin tamamlanmasi beklenilmektedir. 

Bu proje icerisinde amac farkli sektorlerden elde dilmis zaman serileri uzerinden elde edilen faktorler uzerine kurulmus bir classification modeli kurarak benzerlik calismasi yapmaktir.

Mesela ilgilenilen bir hisse senedi X olsun, bunun bulundugu sektor bazli diger hisse senetlerin davranislarindan farkli davrandigini dusunelim. Yani sektor icinde bir artis gozlemlenirken bu hisse senetinde bir hareketlilik olmasin. Dolayisiyla, hangi sektore daha cok benziyor sorusuna cevap verebilirsek, o sektor'un hareketlerine gore bir hipotez kurabiliriz.

Bu proje, asagidaki surecleri kapsayacak:

- Sektorleriin listesine bir web-scraping ile erisilmesi ve verilerin elde edilmesi (`yfinance`, `investpy`, [`quandl`]((https://docs.data.nasdaq.com/v1.0/docs/python-installation)))
- 2005-01-01 yilindan itibaren aylik getirelerden olusan serilerin elde edilmesi
- 3 buyuk sektor uzerinden getirilerin faktorleri(momentum gibi) hesaplanmasi
- Bu momentum serileri uzerinden bir tsfresh ile feature engineering yapilmasi (imputing, encoding, transformation, ve daha fazlasi)
- Yeni elde edilmis feature ve sektor siniflari uzerinden bir model kurulmasi (en iyi model secmesi)
- Diger sektorlerden ornekler alip ayni feature engine yontemleri yaptik sonra hangi sektore benzedigine karar vermek.
- **Bonus** Mesela Real-Estate sektorunde bulunan butun sembollerin tahmini edildikten sonra cogunluk hangi sektore(T,F,H) benzedigi bilgisine erismek.

In [1]:
import yfinance
import pandas as pd
import requests
from bs4 import BeautifulSoup

def fetch_sectors_names():
    url = "https://stockanalysis.com/stocks/industry/sectors/"
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        df=pd.read_html(str(soup.find_all("table")))[0]
    else:
        print(f"Error: Failed to fetch data from page {url}")
        
    return df

def fetch_industry_names():
    url = "https://stockanalysis.com/stocks/industry/all/"
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        df=pd.read_html(str(soup.find_all("table")))[0]
    else:
        print(f"Error: Failed to fetch data from page {url}")
        
    return df

def fetch_data(sectors):
    url = f"https://stockanalysis.com/stocks/sector/{sectors}/"
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        df=pd.read_html(str(soup.find_all("table")))[0]
        df.drop(columns='No.', inplace=True)
    else:
        print(f"Error: Failed to fetch data from page {url}")
        
    return df

In [2]:
sectors=fetch_sectors_names()
indusrty=fetch_industry_names()

C:\Users\sertu\AppData\Local\Temp\ipykernel_7920\4279742626.py:11: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df=pd.read_html(str(soup.find_all("table")))[0]
C:\Users\sertu\AppData\Local\Temp\ipykernel_7920\4279742626.py:22: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df=pd.read_html(str(soup.find_all("table")))[0]


In [3]:
sectors

,Sector Name,Stocks,Market Cap,Div. Yield,PE Ratio,Profit Margin,1D Change,1Y Change
0,Financials,1384,"9,687.64B",2.41%,14.70,17.75%,0.07%,11.52%
1,Healthcare,1217,"8,168.63B",0.43%,50.93,4.07%,0.15%,7.88%
2,Technology,788,17.72T,0.43%,44.88,13.16%,0.44%,15.25%
3,Industrials,652,"5,478.24B",1.10%,26.71,7.32%,0.54%,18.08%
4,Consumer Discretionary,578,"7,265.16B",0.65%,27.34,5.95%,0.31%,2.66%
5,Real Estate,264,"1,506.23B",4.11%,50.70,8.89%,0.77%,8.20%
6,Materials,263,"2,069.67B",1.55%,19.61,8.68%,-0.30%,1.16%
7,Communication Services,260,"5,361.60B",1.08%,28.01,10.40%,-0.01%,2.31%
8,Energy,253,"3,646.08B",2.84%,8.01,12.42%,-0.11%,17.32%
9,Consumer Staples,241,"4,035.53B",1.44%,29.74,4.72%,0.59%,13.61%


## Sektor listerine erismek

Yukarida yazilan fonksyionlar ile hangi sembollerin hangi sektorlerde oldugu bilgisine erisim saglanabilir ve asagidaki betikler yardimi ile `.csv` dosyalarinda saklayabiliriz. Sonrasinda, sektor bazli sembollere ait verileri indirilebilir ve siniflandirilabilir.

In [4]:
fetch_data(sectors='energy').to_csv('../data/stock_sectors/energy.csv')
fetch_data(sectors='financials').to_csv('../data/stock_sectors/financials.csv')
fetch_data(sectors='healthcare').to_csv('../data/stock_sectors/healthcare.csv')
fetch_data(sectors='technology').to_csv('../data/stock_sectors/technology.csv')
fetch_data(sectors='utilities').to_csv('../data/stock_sectors/utilities.csv')
fetch_data(sectors='real-estate').to_csv('../data/stock_sectors/real-estate.csv')
fetch_data(sectors='materials').to_csv('../data/stock_sectors/materials.csv')
fetch_data(sectors='technology').to_csv('../data/stock_sectors/technology.csv')
fetch_data(sectors='industrials').to_csv('../data/stock_sectors/industrials.csv')
fetch_data(sectors='consumer-staples').to_csv('../data/stock_sectors/consumer-staples.csv')
fetch_data(sectors='consumer-discretionary').to_csv('../data/stock_sectors/consumer-discretionary.csv')
fetch_data(sectors='communication-services').to_csv('../data/stock_sectors/communication-services.csv')

C:\Users\sertu\AppData\Local\Temp\ipykernel_7920\4279742626.py:33: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df=pd.read_html(str(soup.find_all("table")))[0]
C:\Users\sertu\AppData\Local\Temp\ipykernel_7920\4279742626.py:33: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df=pd.read_html(str(soup.find_all("table")))[0]
C:\Users\sertu\AppData\Local\Temp\ipykernel_7920\4279742626.py:33: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df=pd.read_html(str(soup.find_all("table")))[0]
C:\Users\sertu\AppData\Local\Temp\ipykernel_7920\4279742626.py:33: FutureWarning: Passing literal html to 'read_html' is deprecated and will b

In [5]:
finance = pd.read_csv('../data/stock_sectors/healthcare.csv')
finance

,Unnamed: 0,Symbol,Company Name,Market Cap,% Change,Volume,Revenue
0,0,LLY,Eli Lilly and Company,716.59B,1.55%,885232,34.12B
1,1,NVO,Novo Nordisk A/S,590.69B,0.63%,1875473,34.41B
2,2,UNH,UnitedHealth Group Incorporated,452.50B,-0.28%,571938,371.62B
3,3,JNJ,Johnson & Johnson,381.18B,-0.54%,1677998,85.16B
4,4,ABBV,AbbVie Inc.,314.19B,0.58%,1670155,54.32B
...,...,...,...,...,...,...,...
1212,1212,MOTS,"Motus GI Holdings, Inc.",977.27K,2.12%,39330,364.00K
1213,1213,REVB,"Revelation Biosciences, Inc.",955.34K,0.21%,26950,-
1214,1214,NBY,"NovaBay Pharmaceuticals, Inc.",914.10K,-1.29%,345648,14.64M
1215,1215,GRI,"GRI Bio, Inc.",746.78K,-1.24%,95247,-








## Verileri çekiyoruz 
## -Aylık getirilerini hesaplıyoruz





In [ ]:
import pandas as pd
import yfinance as yf

# Sektörler ve dosya yolları
sector_files = {
    'financials': '../data/stock_sectors/financials.csv',
    'technology': '../data/stock_sectors/technology.csv',
    'healthcare': '../data/stock_sectors/healthcare.csv'
}

# Her sektör için boş ticker listesi oluştur
ticker_lists = {sector: [] for sector in sector_files}

# Her dosya için sembolleri çek ve listeye ekle
for sector, file_path in sector_files.items():
    df = pd.read_csv(file_path, usecols=[1])  # Yalnızca sembollerin bulunduğu sütunu oku
    tickers = df.iloc[:, 0].tolist()  # DataFrame'den liste oluştur
    ticker_lists[sector].extend(tickers)  # Sektöre ait listeye ekle

# Listeyi temizle: sadece string değerlerin kalmasını sağlar
ticker_f = [ticker for ticker in ticker_lists['financials'] if isinstance(ticker, str)]
ticker_t = [ticker for ticker in ticker_lists['technology'] if isinstance(ticker, str)]
ticker_h = [ticker for ticker in ticker_lists['healthcare'] if isinstance(ticker, str)]

# Sektörlere göre veri indirme ve işleme
data_f = yf.download(ticker_f, start='2005-01-01')
data_t = yf.download(ticker_t, start='2005-01-01')
data_h = yf.download(ticker_h, start='2005-01-01')

# Ay sonu kapanış fiyatlarını al ve aylık getirileri hesapla
data_close_f = data_f['Adj Close'].resample('M').last().pct_change() + 1
data_close_t = data_t['Adj Close'].resample('M').last().pct_change() + 1
data_close_h = data_h['Adj Close'].resample('M').last().pct_change() + 1

# `data_close_f`, `data_close_t`, ve `data_close_h` şimdi her bir sektör için aylık getirileri içerir.


[*********************100%%**********************]  997 of 997 completed

8 Failed downloads:
['CRD.A', 'CRD.B', 'DYCQ', 'BRK.B', 'DISA', 'AGM.A', 'BNRE.A', 'LEGT']: Exception('%ticker%: No timezone found, symbol may be delisted')


## 3 Aylık momentum faktörleri hesaplıyoruz

In [7]:
import numpy as np
# 3 aylik momentum faktorleri
def get_rolling_ret(data, n): 
    return data.rolling(n).apply(np.prod)

calisilacak_veri_f = get_rolling_ret(data_close_f, 3)
calisilacak_veri_t = get_rolling_ret(data_close_t, 3)
calisilacak_veri_h = get_rolling_ret(data_close_h, 3)

In [8]:
#Momentumdan gelen boşlukları dolduruyoruz
calisilacak_veri_h.ffill(inplace=True)
calisilacak_veri_h.bfill(inplace=True)

calisilacak_veri_t.ffill(inplace=True)
calisilacak_veri_t.bfill(inplace=True)

calisilacak_veri_f.ffill(inplace=True)
calisilacak_veri_f.bfill(inplace=True)

In [9]:
calisilacak_veri_h

Ticker,A,AADI,ABBV,ABCL,ABEO,ABIO,ABOS,ABSI,ABT,ABUS,...,ZLAB,ZNTL,ZOM,ZTEK,ZTS,ZURA,ZVRA,ZVSA,ZYME,ZYXI
Date,,,,,,,,,,,,,,,,,,,,,
2005-01-31,0.938489,0.795918,1.266726,0.843936,0.929368,0.737975,0.907623,0.515098,1.098264,1.333333,...,0.786296,0.982906,0.876827,0.388889,0.958561,0.549598,1.523551,1.006079,0.563846,1.000000
2005-02-28,0.938489,0.795918,1.266726,0.843936,0.929368,0.737975,0.907623,0.515098,1.098264,1.333333,...,0.786296,0.982906,0.876827,0.388889,0.958561,0.549598,1.523551,1.006079,0.563846,1.000000
2005-03-31,0.938489,0.795918,1.266726,0.843936,0.929368,0.737975,0.907623,0.515098,1.098264,1.333333,...,0.786296,0.982906,0.876827,0.388889,0.958561,0.549598,1.523551,1.006079,0.563846,1.000000
2005-04-30,0.938489,0.795918,1.266726,0.843936,0.929368,0.737975,0.907623,0.515098,1.098264,1.333333,...,0.786296,0.982906,0.876827,0.388889,0.958561,0.549598,1.523551,1.006079,0.563846,1.000000
2005-05-31,1.000416,0.795918,1.266726,0.843936,0.817869,0.954955,0.907623,0.515098,1.054981,1.333333,...,0.786296,0.982906,0.876827,0.388889,0.958561,0.549598,1.523551,1.006079,0.563846,1.034483
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-30,1.055588,0.834921,0.968903,0.864220,1.083522,0.887805,0.398671,0.932961,1.013508,1.009852,...,1.071457,0.423569,0.908108,0.899281,0.927353,0.704942,0.938856,0.457747,1.225138,1.189610
2023-12-31,1.243337,0.417355,1.039648,1.241304,1.190024,0.845771,0.925301,3.181818,1.136500,1.231527,...,1.124229,0.755234,1.197605,0.864000,1.134441,0.707576,1.358921,0.197802,1.638801,1.361250
2024-01-31,1.258586,0.420091,1.164471,1.243309,1.065445,0.888268,1.700535,3.046154,1.196721,1.295082,...,0.855952,0.724328,0.797619,1.014815,1.196242,0.606383,1.292411,0.198606,1.544160,1.333333


# Verileri Tsfresh için Uygun hale getiriyoruz

In [10]:
calisilacak_veri_h = calisilacak_veri_h.reset_index()
calisilacak_veri_t = calisilacak_veri_t.reset_index()
calisilacak_veri_f = calisilacak_veri_f.reset_index()

In [11]:

calisilacak_veri_h = pd.melt(calisilacak_veri_h, id_vars=['Date'], var_name='Ticker', value_name='Value')
calisilacak_veri_t = pd.melt(calisilacak_veri_t, id_vars=['Date'], var_name='Ticker', value_name='Value')
calisilacak_veri_f = pd.melt(calisilacak_veri_f, id_vars=['Date'], var_name='Ticker', value_name='Value')

In [12]:
calisilacak_veri_f

,Date,Ticker,Value
0,2005-01-31,AACI,1.002351
1,2005-02-28,AACI,1.002351
2,2005-03-31,AACI,1.002351
3,2005-04-30,AACI,1.002351
4,2005-05-31,AACI,1.002351
...,...,...,...
230302,2023-11-30,ZLS,1.013346
230303,2023-12-31,ZLS,1.012311
230304,2024-01-31,ZLS,1.016068
230305,2024-02-29,ZLS,1.011289


In [13]:
calisilacak_veri_f.isna().sum()

Date         0
Ticker       0
Value     4620
dtype: int64

## Boş Kalan Verileri dolduruyoruz

In [14]:
calisilacak_veri_h['Value'].ffill(inplace=True)
calisilacak_veri_h['Value'].bfill(inplace=True)

calisilacak_veri_t['Value'].ffill(inplace=True)
calisilacak_veri_t['Value'].bfill(inplace=True)

calisilacak_veri_f['Value'].ffill(inplace=True)
calisilacak_veri_f['Value'].bfill(inplace=True)


In [15]:
calisilacak_veri_f.isna().sum()

Date      0
Ticker    0
Value     0
dtype: int64

# TSFresh İle Feature Extraction 

In [16]:
import tsfresh
from tsfresh.utilities.dataframe_functions import impute
df_features_h = tsfresh.extract_features(calisilacak_veri_h, column_id='Ticker', column_sort='Date',impute_function=impute, 
                                       default_fc_parameters=tsfresh.feature_extraction.ComprehensiveFCParameters())
df_features_t = tsfresh.extract_features(calisilacak_veri_t, column_id='Ticker', column_sort='Date',impute_function=impute,  
                                       default_fc_parameters=tsfresh.feature_extraction.ComprehensiveFCParameters())
df_features_f = tsfresh.extract_features(calisilacak_veri_f, column_id='Ticker', column_sort='Date',impute_function=impute,  
                                       default_fc_parameters=tsfresh.feature_extraction.ComprehensiveFCParameters())

Feature Extraction: 100%|██████████| 20/20 [00:55<00:00,  2.76s/it]


## TSfresh Sayısal değerler ile çalıştığı için sektörelere ayırıp sayılar veriyorum
**Finans = 0
Sağlık = 1
Teknoloji = 2**

In [17]:
df_features_h['Sector'] = 1
df_features_f['Sector'] = 0
df_features_t['Sector'] = 2

In [18]:
df_features_h

,Value__variance_larger_than_standard_deviation,Value__has_duplicate_max,Value__has_duplicate_min,Value__has_duplicate,Value__sum_values,Value__abs_energy,Value__mean_abs_change,Value__mean_change,Value__mean_second_derivative_central,Value__median,...,Value__fourier_entropy__bins_10,Value__fourier_entropy__bins_100,Value__permutation_entropy__dimension_3__tau_1,Value__permutation_entropy__dimension_4__tau_1,Value__permutation_entropy__dimension_5__tau_1,Value__permutation_entropy__dimension_6__tau_1,Value__permutation_entropy__dimension_7__tau_1,Value__query_similarity_count__query_None__threshold_0.0,Value__mean_n_absolute_max__number_of_maxima_7,Sector
A,0.0,0.0,0.0,1.0,240.299798,254.865679,0.088426,0.000514,-0.000040,1.040310,...,0.998549,2.652565,1.726194,2.922519,4.100086,4.933076,5.284387,0.0,1.353611,1
AADI,0.0,0.0,0.0,1.0,193.165765,171.685780,0.072917,0.001146,0.001495,0.795918,...,0.925396,2.655736,0.926536,1.378403,1.714893,1.850528,1.881803,0.0,1.756573,1
ABBV,0.0,0.0,0.0,1.0,264.108940,306.485018,0.048416,-0.000490,-0.000180,1.206161,...,0.185847,1.283031,1.312042,2.164593,2.903934,3.324162,3.481045,0.0,1.301444,1
ABCL,0.0,0.0,0.0,1.0,195.904123,168.197848,0.026880,-0.000110,-0.000553,0.843936,...,1.058738,2.897530,0.467640,0.681315,0.850227,0.920157,0.957504,0.0,1.245521,1
ABEO,0.0,0.0,0.0,1.0,233.692038,304.060585,0.285370,0.002746,0.000269,0.894904,...,0.708288,2.350879,1.692235,2.882232,3.948556,4.742494,5.186501,0.0,2.903198,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZURA,0.0,0.0,1.0,1.0,129.177631,73.071447,0.006838,0.000528,-0.000191,0.549598,...,0.099050,0.099050,0.162169,0.196916,0.197646,0.198381,0.199123,0.0,0.841348,1
ZVRA,0.0,0.0,0.0,1.0,294.911655,413.757157,0.117049,-0.002625,-0.001083,1.523551,...,0.267826,1.263296,1.236937,1.885652,2.405750,2.658420,2.780447,0.0,2.044759,1
ZVSA,0.0,0.0,0.0,1.0,222.684210,221.552404,0.019745,-0.000268,0.001006,1.006079,...,0.099050,0.099050,0.285026,0.418259,0.537646,0.573739,0.603924,0.0,1.039811,1
ZYME,0.0,0.0,0.0,1.0,169.706110,145.188515,0.067921,0.002049,-0.000696,0.563846,...,0.197798,0.837921,1.016238,1.543692,1.925024,2.084669,2.122884,0.0,1.696534,1


In [19]:
df_features_t

,Value__variance_larger_than_standard_deviation,Value__has_duplicate_max,Value__has_duplicate_min,Value__has_duplicate,Value__sum_values,Value__abs_energy,Value__mean_abs_change,Value__mean_change,Value__mean_second_derivative_central,Value__median,...,Value__fourier_entropy__bins_10,Value__fourier_entropy__bins_100,Value__permutation_entropy__dimension_3__tau_1,Value__permutation_entropy__dimension_4__tau_1,Value__permutation_entropy__dimension_5__tau_1,Value__permutation_entropy__dimension_6__tau_1,Value__permutation_entropy__dimension_7__tau_1,Value__query_similarity_count__query_None__threshold_0.0,Value__mean_n_absolute_max__number_of_maxima_7,Sector
AAOI,0.0,0.0,0.0,1.0,299.195935,450.295622,0.179489,-0.003650,-0.001189,1.501000,...,0.436326,1.858275,1.354508,2.158078,2.847702,3.209291,3.313285,0.0,2.979614,2
AAPL,0.0,0.0,0.0,1.0,249.517819,276.534424,0.099815,-0.000106,-0.000086,1.074274,...,0.905477,2.440645,1.672206,2.802543,3.928630,4.812412,5.261252,0.0,1.498649,2
ACIW,0.0,0.0,0.0,1.0,237.738933,249.715680,0.101628,0.000291,-0.000410,1.018328,...,1.419138,3.185995,1.754605,2.979052,4.123634,4.905926,5.280552,0.0,1.409494,2
ACLS,0.0,0.0,0.0,1.0,246.311850,284.362721,0.180450,0.000070,-0.000129,1.049360,...,0.969509,2.629766,1.731960,2.940872,4.033680,4.900525,5.278226,0.0,1.896721,2
ACMR,0.0,0.0,0.0,1.0,246.546434,282.087445,0.102518,0.001801,-0.000953,1.007408,...,0.976402,2.667033,0.901049,1.346821,1.730790,1.930754,1.995403,0.0,2.282014,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZFOX,0.0,0.0,0.0,1.0,223.634889,219.914608,0.021015,0.001372,-0.000933,0.983202,...,0.099050,0.449173,0.523071,0.758813,0.905450,0.975533,0.985280,0.0,1.231532,2
ZI,0.0,0.0,0.0,1.0,199.686512,175.207665,0.033087,0.000087,-0.000664,0.842446,...,0.275969,1.353257,0.641889,0.907005,1.062156,1.107431,1.123808,0.0,1.284014,2
ZM,0.0,0.0,0.0,1.0,288.965487,372.595059,0.035192,-0.001689,-0.000248,1.317925,...,0.447801,1.437835,0.696082,1.069006,1.317596,1.457357,1.535719,0.0,1.859071,2
ZS,0.0,0.0,0.0,1.0,284.606150,358.462014,0.063350,-0.001755,-0.000775,1.273602,...,0.941103,2.708865,0.899047,1.342223,1.711158,1.850489,1.890222,0.0,1.760748,2


In [20]:
df_features_f

,Value__variance_larger_than_standard_deviation,Value__has_duplicate_max,Value__has_duplicate_min,Value__has_duplicate,Value__sum_values,Value__abs_energy,Value__mean_abs_change,Value__mean_change,Value__mean_second_derivative_central,Value__median,...,Value__fourier_entropy__bins_10,Value__fourier_entropy__bins_100,Value__permutation_entropy__dimension_3__tau_1,Value__permutation_entropy__dimension_4__tau_1,Value__permutation_entropy__dimension_5__tau_1,Value__permutation_entropy__dimension_6__tau_1,Value__permutation_entropy__dimension_7__tau_1,Value__query_similarity_count__query_None__threshold_0.0,Value__mean_n_absolute_max__number_of_maxima_7,Sector
AACI,0.0,0.0,0.0,1.0,231.913007,232.846299,0.001145,4.713919e-04,0.000199,1.002351,...,0.099050,0.099050,0.414440,0.576786,0.639563,0.648054,0.650456,0.0,1.039987,0
AACT,0.0,0.0,0.0,1.0,233.635396,236.300973,0.000074,7.382232e-07,-0.000009,1.011325,...,0.099050,0.099050,0.106172,0.140752,0.169470,0.170101,0.170737,0.0,1.014175,0
AAMC,0.0,0.0,0.0,1.0,181.070985,164.044742,0.040672,1.632223e-03,-0.000276,0.728429,...,0.099050,0.325883,0.264151,0.344217,0.413673,0.483574,0.547763,0.0,2.344726,0
AAME,0.0,0.0,0.0,1.0,236.069449,252.188133,0.122835,5.329131e-04,-0.000642,0.995587,...,0.851282,2.517865,1.729708,2.952458,4.124226,4.915772,5.268924,0.0,1.708723,0
AB,0.0,0.0,0.0,1.0,237.497398,250.077824,0.089354,3.431254e-04,-0.000058,1.028884,...,1.098134,2.705139,1.771851,3.039615,4.249980,4.996048,5.279736,0.0,1.416489,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YOTA,0.0,0.0,0.0,1.0,232.792909,234.603799,0.000831,5.257723e-05,-0.000027,1.007231,...,0.099050,0.099050,0.314206,0.413669,0.466073,0.501977,0.509999,0.0,1.025638,0
YRD,0.0,0.0,0.0,1.0,269.136011,339.881094,0.108493,2.226522e-03,-0.000150,1.231746,...,0.655086,2.328048,1.121015,1.720876,2.191788,2.492709,2.559168,0.0,2.375947,0
ZEO,0.0,0.0,0.0,1.0,232.834473,234.901089,0.002513,-2.012571e-03,-0.001037,1.009073,...,0.099050,0.099050,0.406902,0.549867,0.625044,0.641919,0.650456,0.0,1.028798,0
ZION,0.0,0.0,0.0,1.0,235.659431,249.481446,0.113154,-5.417334e-04,-0.000448,1.017652,...,1.178177,2.804548,1.716388,2.917609,4.025949,4.833129,5.211962,0.0,1.506293,0


## Finans Sağlık Teknoloji verilerini birleştiriyorum

In [21]:
df_combination = pd.concat([df_features_h,df_features_t,df_features_f], axis=0)

In [22]:
df_combination

,Value__variance_larger_than_standard_deviation,Value__has_duplicate_max,Value__has_duplicate_min,Value__has_duplicate,Value__sum_values,Value__abs_energy,Value__mean_abs_change,Value__mean_change,Value__mean_second_derivative_central,Value__median,...,Value__fourier_entropy__bins_10,Value__fourier_entropy__bins_100,Value__permutation_entropy__dimension_3__tau_1,Value__permutation_entropy__dimension_4__tau_1,Value__permutation_entropy__dimension_5__tau_1,Value__permutation_entropy__dimension_6__tau_1,Value__permutation_entropy__dimension_7__tau_1,Value__query_similarity_count__query_None__threshold_0.0,Value__mean_n_absolute_max__number_of_maxima_7,Sector
A,0.0,0.0,0.0,1.0,240.299798,254.865679,0.088426,0.000514,-0.000040,1.040310,...,0.998549,2.652565,1.726194,2.922519,4.100086,4.933076,5.284387,0.0,1.353611,1
AADI,0.0,0.0,0.0,1.0,193.165765,171.685780,0.072917,0.001146,0.001495,0.795918,...,0.925396,2.655736,0.926536,1.378403,1.714893,1.850528,1.881803,0.0,1.756573,1
ABBV,0.0,0.0,0.0,1.0,264.108940,306.485018,0.048416,-0.000490,-0.000180,1.206161,...,0.185847,1.283031,1.312042,2.164593,2.903934,3.324162,3.481045,0.0,1.301444,1
ABCL,0.0,0.0,0.0,1.0,195.904123,168.197848,0.026880,-0.000110,-0.000553,0.843936,...,1.058738,2.897530,0.467640,0.681315,0.850227,0.920157,0.957504,0.0,1.245521,1
ABEO,0.0,0.0,0.0,1.0,233.692038,304.060585,0.285370,0.002746,0.000269,0.894904,...,0.708288,2.350879,1.692235,2.882232,3.948556,4.742494,5.186501,0.0,2.903198,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YOTA,0.0,0.0,0.0,1.0,232.792909,234.603799,0.000831,0.000053,-0.000027,1.007231,...,0.099050,0.099050,0.314206,0.413669,0.466073,0.501977,0.509999,0.0,1.025638,0
YRD,0.0,0.0,0.0,1.0,269.136011,339.881094,0.108493,0.002227,-0.000150,1.231746,...,0.655086,2.328048,1.121015,1.720876,2.191788,2.492709,2.559168,0.0,2.375947,0
ZEO,0.0,0.0,0.0,1.0,232.834473,234.901089,0.002513,-0.002013,-0.001037,1.009073,...,0.099050,0.099050,0.406902,0.549867,0.625044,0.641919,0.650456,0.0,1.028798,0
ZION,0.0,0.0,0.0,1.0,235.659431,249.481446,0.113154,-0.000542,-0.000448,1.017652,...,1.178177,2.804548,1.716388,2.917609,4.025949,4.833129,5.211962,0.0,1.506293,0


In [23]:
data = df_combination

In [24]:
df = data


## TSFresh ile Feature_selection yaptım, boş değerleri dolduruyorum.

In [25]:
from tsfresh import select_features
from tsfresh.utilities.dataframe_functions import impute
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
import pandas as pd

# Hedef değişken (target) ve özellikler (features) olarak ayırma
X = df.drop(['Sector'], axis=1)
y = df['Sector']

# tsfresh ile eksik değerlerin doldurulması
X_imputed = impute(X)

# RandomForestClassifier modelini kullanarak feature selection
clf = RandomForestClassifier(n_estimators=100, random_state=0, n_jobs=-1)
clf = clf.fit(X_imputed, y)

# SelectFromModel ile önemli özellikleri seç
selector = SelectFromModel(clf, prefit=True)
X_selected = selector.transform(X_imputed)

# Seçilen özelliklerin DataFrame'e dönüştürülmesi
selected_features_df = pd.DataFrame(X_selected, columns=X_imputed.columns[selector.get_support()])

selected_features_df


C:\Users\sertu\anaconda3\Lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


,Value__mean_abs_change,Value__mean,Value__standard_deviation,Value__variation_coefficient,Value__variance,Value__absolute_sum_of_changes,Value__last_location_of_minimum,Value__percentage_of_reoccurring_values_to_all_values,Value__percentage_of_reoccurring_datapoints_to_all_datapoints,Value__sum_of_reoccurring_values,...,Value__energy_ratio_by_chunks__num_segments_10__segment_focus_7,Value__energy_ratio_by_chunks__num_segments_10__segment_focus_8,Value__ratio_beyond_r_sigma__r_0.5,Value__lempel_ziv_complexity__bins_10,Value__lempel_ziv_complexity__bins_100,Value__permutation_entropy__dimension_3__tau_1,Value__permutation_entropy__dimension_4__tau_1,Value__permutation_entropy__dimension_5__tau_1,Value__permutation_entropy__dimension_6__tau_1,Value__permutation_entropy__dimension_7__tau_1
0,0.088426,1.040259,0.145520,0.139888,0.021176,20.337992,0.207792,0.004386,0.017316,0.938489,...,0.097844,0.105829,0.632035,0.393939,0.632035,1.726194,2.922519,4.100086,4.933076,5.284387
1,0.072917,0.836215,0.209696,0.250767,0.043972,16.770894,0.744589,0.014085,0.696970,0.795918,...,0.127275,0.184904,0.203463,0.203463,0.307359,0.926536,1.378403,1.714893,1.850528,1.881803
2,0.048416,1.143329,0.139908,0.122369,0.019574,11.135647,0.558442,0.007576,0.432900,1.266726,...,0.076863,0.091837,0.839827,0.324675,0.471861,1.312042,2.164593,2.903934,3.324162,3.481045
3,0.026880,0.848070,0.094376,0.111283,0.008907,6.182490,0.848485,0.027027,0.844156,0.843936,...,0.097393,0.083730,0.121212,0.181818,0.220779,0.467640,0.681315,0.850227,0.920157,0.957504
4,0.285370,1.011654,0.541143,0.534910,0.292836,65.635079,0.519481,0.022523,0.060606,5.416443,...,0.066891,0.065983,0.502165,0.307359,0.584416,1.692235,2.882232,3.948556,4.742494,5.186501
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,0.000831,1.007762,0.004195,0.004163,0.000018,0.191113,0.987013,0.052632,0.922078,1.007231,...,0.099461,0.099461,0.069264,0.134199,0.160173,0.314206,0.413669,0.466073,0.501977,0.509999
2996,0.108493,1.165091,0.337505,0.289681,0.113910,24.953312,0.870130,0.010309,0.584416,1.231746,...,0.048050,0.065509,0.294372,0.225108,0.333333,1.121015,1.720876,2.191788,2.492709,2.559168
2997,0.002513,1.007941,0.030690,0.030448,0.000942,0.577923,1.000000,0.040000,0.896104,1.009073,...,0.099698,0.099738,0.034632,0.090909,0.142857,0.406902,0.549867,0.625044,0.641919,0.650456
2998,0.113154,1.020171,0.198136,0.194219,0.039258,26.025338,0.216450,0.004386,0.017316,1.037924,...,0.084745,0.118817,0.480519,0.354978,0.606061,1.716388,2.917609,4.025949,4.833129,5.211962


# TAHMİNLER EN İYİ MODEL SEÇİMİ

## Random Forest ile Tahmin(0.92)

In [26]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

# Veri setini eğitim ve test setlerine ayırma
X_train, X_test, y_train, y_test = train_test_split(selected_features_df, y, test_size=0.2, random_state=42)

# RandomForestClassifier modelini eğitme
clf = RandomForestClassifier(n_estimators=100, random_state=0, n_jobs=-1)
clf.fit(X_train, y_train)

# Test seti üzerinde tahmin yapma
y_pred = clf.predict(X_test)

# Modelin doğruluğunu ve diğer metriklerini hesaplama
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.9166666666666666

Classification Report:
               precision    recall  f1-score   support

           0       0.92      0.96      0.94       185
           1       0.94      0.92      0.93       259
           2       0.87      0.87      0.87       156

    accuracy                           0.92       600
   macro avg       0.91      0.91      0.91       600
weighted avg       0.92      0.92      0.92       600



## Logistic Regression ile Tahmin(0.66)

In [27]:
from sklearn.linear_model import LogisticRegression

# Logistic Regression modelini eğitme
log_reg = LogisticRegression(max_iter=1000,random_state=0)
log_reg.fit(X_train, y_train)

# Tahmin yapma
y_pred_log_reg = log_reg.predict(X_test)

# Performans değerlendirme
accuracy_log_reg = accuracy_score(y_test, y_pred_log_reg)
print(f"Logistic Regression Accuracy: {accuracy_log_reg}")
print("\nLogistic Regression Classification Report:\n", classification_report(y_test, y_pred_log_reg))


Logistic Regression Accuracy: 0.7033333333333334

Logistic Regression Classification Report:
               precision    recall  f1-score   support

           0       0.78      0.84      0.81       185
           1       0.69      0.77      0.73       259
           2       0.60      0.43      0.50       156

    accuracy                           0.70       600
   macro avg       0.69      0.68      0.68       600
weighted avg       0.69      0.70      0.69       600



C:\Users\sertu\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## SVM ile Tahmin (0.70)
## -"StandarScaler" kullandım uzun sürdüğü için

In [28]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


# Öncelikle, verileri ölçeklendirme için StandardScaler örneği oluştur
scaler = StandardScaler()

# Eğitim verisini ölçeklendir
X_train_scaled = scaler.fit_transform(X_train)

# Test verisini ölçeklendir (Eğitim verisine göre ölçeklendirme yapılır)
X_test_scaled = scaler.transform(X_test)

# Ölçeklenmiş veri ile SVM modelini eğitme
svm_clf = SVC(kernel='linear', random_state=0)
svm_clf.fit(X_train_scaled, y_train)

# Tahmin yapma (Ölçeklenmiş test verisi üzerinde)
y_pred_svm = svm_clf.predict(X_test_scaled)

# Performans değerlendirme
accuracy_svm = accuracy_score(y_test, y_pred_svm)
print(f"SVM Accuracy: {accuracy_svm}")
print("\nSVM Classification Report:\n", classification_report(y_test, y_pred_svm))



SVM Accuracy: 0.7033333333333334

SVM Classification Report:
               precision    recall  f1-score   support

           0       0.80      0.82      0.81       185
           1       0.69      0.77      0.73       259
           2       0.59      0.46      0.51       156

    accuracy                           0.70       600
   macro avg       0.69      0.68      0.68       600
weighted avg       0.70      0.70      0.70       600



## Gradient Boosting ile Tahmin.(0.91)

In [29]:
from sklearn.ensemble import GradientBoostingClassifier

# Gradient Boosting modelini eğitme
gb_clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
gb_clf.fit(X_train, y_train)

# Tahmin yapma
y_pred_gb = gb_clf.predict(X_test)

# Performans değerlendirme
accuracy_gb = accuracy_score(y_test, y_pred_gb)
print(f"Gradient Boosting Accuracy: {accuracy_gb}")
print("\nGradient Boosting Classification Report:\n", classification_report(y_test, y_pred_gb))

Gradient Boosting Accuracy: 0.9116666666666666

Gradient Boosting Classification Report:
               precision    recall  f1-score   support

           0       0.92      0.96      0.94       185
           1       0.92      0.91      0.92       259
           2       0.88      0.86      0.87       156

    accuracy                           0.91       600
   macro avg       0.91      0.91      0.91       600
weighted avg       0.91      0.91      0.91       600



# Tahminler 
### -Random Forest ve Gradient Boosting yakın çıkarken 0.91 ile
### -Öbürleri daha aşağıda değerler yakaladı

# Öbür Hisseleri deneme Zamanı şimdi

## -Ayni feature engine yontemleri yaptiktan sonra hangi sektore benzedigine karar vermek

## SHELL FİNANSAL'a daha yakınmış

In [30]:
import pandas as pd
import yfinance as yf
from tsfresh import extract_features, select_features
from tsfresh.utilities.dataframe_functions import impute
from sklearn.ensemble import RandomForestClassifier
import numpy as np

# SHEL hissesi için veri indir
data = yf.download("SHEL", start='2005-01-01')
data_close = data['Adj Close'].resample('M').last().pct_change() + 1

# 3 aylık momentum faktörleri
def get_rolling_ret(data, n): 
    return data.rolling(n).apply(np.prod)

calisilacak_veri = get_rolling_ret(data_close, 3)
calisilacak_veri.ffill(inplace=True)
calisilacak_veri.bfill(inplace=True)
calisilacak_veri = calisilacak_veri.reset_index()
calisilacak_veri = pd.melt(calisilacak_veri, id_vars=['Date'], var_name='Ticker', value_name='Value')

# Feature Extraction
df_features = extract_features(calisilacak_veri, column_id='Ticker', column_sort='Date',  
                                    default_fc_parameters=tsfresh.feature_extraction.ComprehensiveFCParameters())

#Feature Selection

columns_to_drop = [col for col in df_features.columns if col not in selected_features_df.columns]

# Bulunan sütunları çıkar
extracted_features_filtered = df_features.drop(columns=columns_to_drop)

y_pred = clf.predict(extracted_features_filtered)

sektorler = ['Finansal', 'Teknoloji', 'Sağlık']
print(f"SHEL hissesi {sektorler[y_pred[0]]} sektörüne daha yakın.")

[*********************100%%**********************]  1 of 1 completed
Feature Extraction: 100%|██████████| 1/1 [00:05<00:00,  5.59s/it]

SHEL hissesi Finansal sektörüne daha yakın.


## GOLD hissesi sağlık sektörüne daha benzermiş

In [32]:
# GOLD hissesi için veri indir
data = yf.download("GOLD", start='2005-01-01')
data_close = data['Adj Close'].resample('M').last().pct_change() + 1

# 3 aylık momentum faktörleri
def get_rolling_ret(data, n): 
    return data.rolling(n).apply(np.prod)

calisilacak_veri = get_rolling_ret(data_close, 3)
calisilacak_veri.ffill(inplace=True)
calisilacak_veri.bfill(inplace=True)
calisilacak_veri = calisilacak_veri.reset_index()
calisilacak_veri = pd.melt(calisilacak_veri, id_vars=['Date'], var_name='Ticker', value_name='Value')

# Feature Extraction
df_features = extract_features(calisilacak_veri, column_id='Ticker', column_sort='Date',  
                                    default_fc_parameters=tsfresh.feature_extraction.ComprehensiveFCParameters())

#Feature Selection

columns_to_drop = [col for col in df_features.columns if col not in selected_features_df.columns]

# Bulunan sütunları çıkar
extracted_features_filtered = df_features.drop(columns=columns_to_drop)

y_pred = clf.predict(extracted_features_filtered)

sektorler = ['Finansal', 'Teknoloji', 'Sağlık']
print(f"GOLD hissesi {sektorler[y_pred[0]]} sektörüne daha yakın.")

[*********************100%%**********************]  1 of 1 completed
Feature Extraction: 100%|██████████| 1/1 [00:05<00:00,  5.59s/it]

GOLD hissesi Sağlık sektörüne daha yakın.
